In [ ]:
from pipeline2 import PipelineLevels, AcquisitionPipeline, DefaultNameHandler, HDF5RichData, HDF5DataStore
from pipeline2.taskgeneration import (AcquisitionTaskGenerator, JSONFileConfigLoader, DefaultFOVSettingsGenerator,
                                      DefaultScanOffsetsSettingsGenerator,
                                      DefaultLocationRemover, DefaultStageOffsetsSettingsGenerator,
                                      SpiralOffsetGenerator, NewestSettingsSelector, NewestDataSelector,
                                      BoundingBoxLocationGrouper, DefaultScanModeSettingsGenerator)
from pipeline2.detection import LegacySpotPairFinder, SimpleSingleChannelSpotDetector
from pipeline2.imspector import ImspectorConnection
from pipeline2.stoppingcriteria import TimedStoppingCriterion

import specpy as sp

import numpy as np
from matplotlib import pyplot as plt


import h5py as h5


#BoundingBoxLocationGrouper()


In [ ]:
# plotting params
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

# path to generic golbead config_path
gb_config_path = 'C:/Users/RESOLFT/Desktop/config_json/goldbeads_overview.json'

# imspector object
im = sp.Imspector()



pll = PipelineLevels('overview', 'detail', 'more_detail')


# overview task generator: same settings every time
atg_overview = (AcquisitionTaskGenerator(pll.overview, 
                    DefaultLocationRemover(
                        JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/goldbeads_overview.json'])
                    )
                )
                .withDelay(.5))

name_handler = DefaultNameHandler('C:/Users//RESOLFT/Desktop/TEST_GEN/', pll)

# abuse name_handler to get h5 filename
h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

with h5.File(h5file_name, mode='a') as h5fd:
    data_store = HDF5DataStore(h5fd, pll)

    # pipeline
    pl = (AcquisitionPipeline('1')
            .withImspectorConnection(ImspectorConnection(im))
            .withPipelineLevels(pll)
            .withNameHandler(name_handler)
            .withAddedStoppingCondition(TimedStoppingCriterion(200)))
    
    # attach h5 data store
    pl.withDataStorage(data_store)

    # init detector
    detector = SimpleSingleChannelSpotDetector(NewestDataSelector(pl, pll.overview), 2, 2).withPlotDetections().withVerbose()

    # detail task generation: settings from last overview + new FOV + grouped detections
    atg_detail = AcquisitionTaskGenerator(pll.detail,
                             NewestSettingsSelector(pl, pll.overview),
                             DefaultFOVSettingsGenerator([[1e-5, 1e-5, 0]], [[5e-8, 5e-8, 0]]),
                             DefaultScanOffsetsSettingsGenerator(BoundingBoxLocationGrouper(detector, [1e-5, 1e-5, 1e-5]).withVerbose()))

    # overview callback: re-add overview
    pl.withCallbackAtLevel(atg_overview, pll.overview)
    # overview callback2: detect & do detail
    pl.withCallbackAtLevel(atg_detail, pll.overview)


    # another detector in detail
    detector2 = SimpleSingleChannelSpotDetector(NewestDataSelector(pl, pll.detail), 5, 0.1, 0, 1.2).withPlotDetections().withVerbose()

    # detail task generation: settings from last overview + new FOV + grouped detections
    atg_detail_more = AcquisitionTaskGenerator(pll.more_detail,
                             NewestSettingsSelector(pl, pll.detail),
                             DefaultScanModeSettingsGenerator(['xy', 'xz', 'yz'], False),
                             DefaultFOVSettingsGenerator([[5e-7, 5e-7, 5e-7]], [[5e-9, 5e-9, 2e-8]]),
                             DefaultScanOffsetsSettingsGenerator(BoundingBoxLocationGrouper(detector2, [5e-7, 5e-7, 5e-7]).withVerbose())
                             )

    # dummy callback: print & plot detections in detail img
    #pl.withCallbackAtLevel(lambda x: print(detector2.get_locations()), pll.detail)

    # overview callback2: detect & do detail
    pl.withCallbackAtLevel(atg_detail_more, pll.detail)

    # call overview atg once to add first measurement to pipeline
    atg_overview(pl)

    # GO
    pl.run()


In [ ]:
import json
im = sp.Imspector()
par = im.active_measurement().parameters('')
print(json.dumps(par['Measurement']['axes']['scan_axes'], indent=2))

mf = 'xy'
par['ExpControl']['scan']['range']['mode'] = gen_mode_flag(mf)

par['Measurement']['axes']['scan_axes'] = ['ExpControl {}'.format(mf[i].upper()) if i < len(mf) else "None" for i in range(4)]

im.active_measurement().set_parameters('', par)

In [ ]:
modes = {
    'xyz' : 528,
    'xyt' : 784,
    'x' : 1360,
    'y' : 1361,
    'z' : 1362,
    't' : 1363,
    'xy' : 1296,
    'yx' : 1281,
    
    
    
    
    
}

In [ ]:
int('001100000001', 2)

In [ ]:
def gen_mode_flag(mode_str):

    _mode_vals = {
    'x' : 0,
    'y' : 1,
    'z' : 2,
    't' : 3
    }
    
    if len(mode_str) > 4:
        return None
    res = 0
    for _ in range(3 - len(mode_str)):
        print(res)
        res = (res + 1) << 2
        res = (res + 1) << 2
    for i, c in enumerate(reversed(mode_str)):
        res = res << 2
        res = res + _mode_vals[c]
        if not i == len(mode_str) - 1:
            res = res << 2
    return res

gen_mode_flag('xy')

In [ ]:
from skimage.feature import peak_local_max

peak_local_max(np.array([[1,0,0,0], [0,0,0,1]]), min_distance=1)